In [3]:
# Do all imports and installs here
from pyspark.sql.functions import udf
from pyspark.sql.session import SparkSession
import datetime
import pandas as pd
import pyspark.sql.functions as F
import psycopg2


In [4]:
spark = SparkSession.builder\
                    .config("spark.jars.packages","org.apache.hadoop:hadoop-aws:2.7.0")\
                    .appName("Project: i94")\
                    .getOrCreate()

In [5]:
# Read csv file(s)
csv_i94addr = pd.read_csv('D:/Capstone-Project/Project-Workspace/i94addr.csv', quotechar="'")

In [6]:
csv_i94addr.head()

,I94ADDR,ADDR_NAME
0,AL,ALABAMA
1,AK,ALASKA
2,AZ,ARIZONA
3,AR,ARKANSAS
4,CA,CALIFORNIA


In [7]:
sp_i94addr = spark.createDataFrame(csv_i94addr)

In [8]:
sp_i94addr.coalesce(1).write.format('json').mode('overwrite').save('D:/capstone/solarhenge/dim_addr')


In [4]:
conn = psycopg2.connect("host=127.0.0.1 dbname=i94 user=i94user password=Passw0rd")
cur = conn.cursor()

delete_from_dim_addr = ("""
delete from dim_addr;
""")

insert_into_dim_addr = ("""
insert into dim_addr
(i94addr
,addr_name
) values 
(%s
,%s
);
""")

# a function to insert records
def insert_dim_addr_records(cur, addr_df):
    for i, row in addr_df.iterrows():
        #print('i =',i)
        #print('row =',row)
        cur.execute(insert_into_dim_addr,row)

# delete records
cur.execute(delete_from_dim_addr)

# insert records
insert_dim_addr_records(cur, csv_i94addr)

conn.commit()
conn.close()